In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from WindPy import w
from sklearn.linear_model import LinearRegression
import datetime
%matplotlib inline

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
end_date = datetime.datetime.today()
start_date = end_date - datetime.timedelta(200)

# 获取股票指数历史收益率

In [4]:
# codes = 'CI005917.WI,CI005919.WI,CI005018.WI,CI005916.WI' # 金融地产、消费、医药、科技
codes = '000934.SH,000931.CSI,000933.SH,000935.SH' # 中证金融地产、可选消费、必选医药、信息技术

In [5]:
data = w.wsd(codes, 'close', start_date, end_date)

In [6]:
df = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [7]:
rdf = df.pct_change().dropna()

# 获取股票基金历史收益率

In [8]:
fund_df = pd.read_excel(u'../data/股票基金池.xlsx')
# fund_df = pd.read_excel(u'../data/嘉实股票基金池.xlsx')

In [9]:
fund_df

,代码,名称,市值占净值
0,000176.OF,嘉实沪深300增强,0.200000
1,160716.OF,嘉实基本面50指数(LOF)A,0.200000
2,005662.OF,嘉实金融精选A,0.199999
3,070019.OF,嘉实价值优势,0.199999
4,000008.OF,嘉实中证500ETF联接A,0.199999
5,000711.OF,嘉实医疗保健,0.199998
6,003053.OF,嘉实文体娱乐A,0.199997
7,070011.OF,嘉实策略增长,0.199997
8,070021.OF,嘉实主题新动力,0.199996


In [10]:
codes = fund_df[u'代码'].tolist()

In [11]:
codes

[u'000176.OF',
 u'160716.OF',
 u'005662.OF',
 u'070019.OF',
 u'000008.OF',
 u'000711.OF',
 u'003053.OF',
 u'070011.OF',
 u'070021.OF']

In [12]:
data = w.wsd(codes, 'nav', start_date, end_date)

In [13]:
fdf = pd.DataFrame(np.array(data.Data).T, index=data.Times, columns=data.Codes)

In [14]:
fdf = fdf.dropna(axis=1)

In [15]:
rfdf = fdf.pct_change().dropna()

In [16]:
codes = rfdf.columns.tolist()

In [17]:
print rfdf.shape
print rdf.shape

(135, 9)
(135, 4)


# 非负线性优化求解

In [18]:
# decomp = pd.DataFrame(index=codes, columns=[u'金融地产', u'消费', u'医药', u'科技'])
decomp = pd.DataFrame(index=codes, columns=[u'金融地产', u'可选消费', u'必选医药', u'科技'])
decomp.index.name = u'基金代码'

In [19]:
def objective(coef, args):
    n = coef.size
    X, y = args[0], args[1]
    target = np.dot(X, coef)
#     error = np.sum(np.square(target - y))
    error = np.sum(np.square(target - y)) + np.linalg.norm(coef, 1) # lasso error
    return error / n

def decomposition(code):
    TOLERANCE = 1e-10
    n = rdf.shape[1]
    init_weights = np.array([1. / n] * n)
    X = rdf.values
    y = rfdf[code].values
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})
    
    opt_res = minimize(fun=objective,
                       x0=init_weights,
                       args=[X, y],
                       constraints=constraints,
                       tol=TOLERANCE,
                       options={'disp': True})
    
    return opt_res.x

In [20]:
decomposition(codes[0])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250193963914
            Iterations: 20
            Function evaluations: 120
            Gradient evaluations: 20


array([0.57039419, 0.16012622, 0.21327435, 0.05620524])

In [21]:
for c in codes:
    weight = decomposition(c)
    decomp.loc[c] = weight

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250193963914
            Iterations: 20
            Function evaluations: 120
            Gradient evaluations: 20
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250268483248
            Iterations: 17
            Function evaluations: 102
            Gradient evaluations: 17
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250318883414
            Iterations: 18
            Function evaluations: 108
            Gradient evaluations: 18
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250564490113
            Iterations: 17
            Function evaluations: 102
            Gradient evaluations: 17
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.250498729745
            Iterations: 21
            Function evaluations: 1

In [22]:
decomp[u'基金名称'] = fund_df[fund_df[u'代码'].isin(codes)][u'名称'].tolist()

In [23]:
if u'市值占净值' in fund_df.columns:
    decomp[u'市值占净值'] = fund_df[fund_df[u'代码'].isin(codes)][u'市值占净值'].tolist()

In [24]:
decomp = decomp[decomp.columns[4:].tolist() + decomp.columns[:4].tolist()]

In [25]:
decomp

,基金名称,市值占净值,金融地产,可选消费,必选医药,科技
基金代码,,,,,,
000176.OF,嘉实沪深300增强,0.200000,0.570394,0.160126,0.213274,0.0562052
160716.OF,嘉实基本面50指数(LOF)A,0.200000,0.796087,0.181527,0.0223862,-3.75405e-18
005662.OF,嘉实金融精选A,0.199999,0.958662,-1.8052e-17,0.0413376,1.05439e-17
070019.OF,嘉实价值优势,0.199999,0.392395,0.252607,0.336242,0.0187557
000008.OF,嘉实中证500ETF联接A,0.199999,0.289526,0.193893,0.159883,0.356697
000711.OF,嘉实医疗保健,0.199998,-5.52284e-17,8.32667e-17,1,-3.76595e-17
003053.OF,嘉实文体娱乐A,0.199997,0.140588,0.0544448,0.158305,0.646663
070011.OF,嘉实策略增长,0.199997,0.325585,0.322871,0.122578,0.228967
070021.OF,嘉实主题新动力,0.199996,0.0148935,0.1423,0.143251,0.699555


In [26]:
if u'市值占净值' in fund_df.columns:
    decomp = decomp.append(pd.DataFrame([['', '',
                           np.dot(decomp[u'金融地产'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(),
                           np.dot(decomp[u'可选消费'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(),
                           np.dot(decomp[u'必选医药'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(),
                           np.dot(decomp[u'科技'], fund_df[u'市值占净值']) / fund_df[u'市值占净值'].sum(), 
                           ]], index=[u'组合'], columns=decomp.columns))

In [27]:
decomp.to_excel(u'../data/股票基金收益率分解.xlsx')
# decomp.to_excel(u'../data/嘉实股票基金收益率分解.xlsx')